In [1]:
import glob
import os

import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import seaborn as sns

/Users/mjain/DoSomething/data/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
def load_connection(
        user_str='DB_USER',
        pwd_str='DB_PW',
        db_str='DB_NAME',
        host_str='DB_HOST'
):
    user = os.environ.get(user_str)
    pwd = os.environ.get(pwd_str)
    db = os.environ.get(db_str)
    host = os.environ.get(host_str)
    return psycopg2.connect(database=db, user=user, password=pwd, host=host)

conn = load_connection()

### Data prep

In [3]:
df = pd.read_csv('../../Downloads/ds_ambassadors/final_032619.csv')

In [4]:
df = df[~(df['NSID'].duplicated(keep=False))]

In [5]:
# Number of users in each group
df.groupby('group').agg({'NSID': lambda x: len(x.unique())})

,NSID
group,
1,66
2,63
3,59
4,63


In [6]:
df.rename(columns={'NSID': 'northstar_id'}, inplace=True)

### Engagement

Number of interactions with DS platform using count of actions in MEL

In [7]:
users = u.sql_stringify_list(df['northstar_id'].unique())

In [8]:
mel_sql = '''
SELECT *
FROM member_event_log
WHERE northstar_id in ({})
AND "timestamp" >= '2019-03-01'
'''.format(users)

df_mel = pd.read_sql(mel_sql, conn)

In [9]:
df_mel.describe()

,event_id,northstar_id,timestamp,action_type,action_id,source,action_serial_id,channel,type,first_action_month
count,1101,1101,1101,1101,1101,703,1101,1101,1101,1101
unique,1101,134,1035,7,7,7,704,3,1,148
top,a2f0db0be558e9d34a0aaad7c24888f7,5c4d1b5ece48d400bd1067e1,2019-03-07 03:02:22+00:00,clicked_link,7,4,0,web,Returning,2019-03-22 23:38:48+00:00
freq,1,42,2,390,390,334,398,613,1101,41


In [10]:
df_mel_merge = pd.merge(df_mel, df[['northstar_id', 'group']], on=['northstar_id'], how='left')

In [11]:
len(df_mel), len(df_mel_merge)

(1101, 1101)

Find monthly number of active users.

In [12]:
df_mel_merge.groupby('group').agg({'northstar_id': lambda x: len(x.unique())})

,northstar_id
group,
1,26
2,32
3,36
4,40


Find number of active users, number of actions, and number of signups week-over-week.

In [13]:
dt_index = df_mel_merge.set_index(['timestamp']).resample('W').sum().index

In [14]:
def groupby_fxns(df):
    grouped_result = {}
    grouped_result['num_active_users'] = len(df['northstar_id'].unique())
    grouped_result['num_actions'] = len(df)
    grouped_result['num_signups'] = len(df[df['action_type'] == 'signup'])
    return pd.Series(grouped_result, index=grouped_result.keys())

In [15]:
# create multiindex
idx = pd.MultiIndex.from_product([dt_index, df_mel_merge['group'].unique()])

In [16]:
df_final = df_mel_merge.set_index('timestamp').groupby(
    [pd.Grouper(freq='W'), 'group']).apply(groupby_fxns).reindex(idx).fillna(0)

In [17]:
df_final

num_active_users  num_actions  num_signups
2019-03-03 00:00:00+00:00 3               0.0          0.0          0.0
                          4               3.0         17.0          3.0
                          2               5.0         12.0          2.0
                          1               4.0         12.0          4.0
2019-03-10 00:00:00+00:00 3              28.0        169.0         24.0
                          4              25.0         98.0         17.0
                          2              21.0        116.0         18.0
                          1              14.0         67.0         14.0
2019-03-17 00:00:00+00:00 3              13.0         63.0          9.0
                          4              17.0         76.0         10.0
                          2              16.0         44.0          6.0
                          1              11.0         32.0          7.0
2019-03-24 00:00:00+00:00 3              10.0         29.0          5.0
                          4              24.0         78.0          6.0
                          2              17.0         36.0          1.0
                          1              10.0         28.0          2.0
2019-03-31 00:00:00+00:00 3               9.0         38.0          2.0
                          4              18.0         65.0          4.0
                          2               9.0         32.0          2.0
                          1              11.0         31.0          3.0
2019-04-07 00:00:00+00:00 3               3.0         12.0          2.0
                          4               4.0         11.0          2.0
                          2               5.0         18.0          5.0
                          1               6.0         17.0          3.0

### Impact

In [18]:
rb_sql = '''
SELECT s.*, p.quantity, r.post_id, r.reportback_volume
FROM signups s
LEFT JOIN (SELECT signup_id, id, quantity FROM posts) p
ON s.id = p.signup_id
LEFT JOIN reportbacks r
ON s.id = r.signup_id
AND p.id = r.post_id
WHERE s.northstar_id in ({})
AND s.created_at >= '2019-03-01'
'''

df_rbs = pd.read_sql(rb_sql.format(users), conn)

In [19]:
df_rbs_merge = pd.merge(df_rbs, df[['northstar_id', 'group']], on=['northstar_id'], how='left')

In [20]:
# Confirm signups align with MEL
df_rbs_merge.groupby(['group']).agg({'northstar_id': lambda x: len(x.unique())})

,northstar_id
group,
1,15
2,15
3,19
4,20


In [21]:
# Shows why signups don't align with MEL because MEL does not include voter-reg etc.
df_rbs_merge[df_rbs_merge['group'] == 4].sort_values('created_at')['source'].value_counts()

phoenix-next     42
sms               5
rock-the-vote     1
Name: source, dtype: int64

Find number of reportbacks and total quantity (if applicable).

In [22]:
def groupby_rb_fxns(df):
    grouped_result = {}
    grouped_result['num_rbs'] = len(df['post_id'].unique())
    grouped_result['total_quantity'] = sum(df['quantity'])
    return pd.Series(grouped_result, index=grouped_result.keys())

In [23]:
df_final_rbs = df_rbs_merge.set_index('created_at').groupby(
    [pd.Grouper(freq='W'), 'group']).apply(groupby_rb_fxns).reindex(idx).fillna(0)

In [24]:
df_final_rbs

num_rbs  total_quantity
2019-03-03 00:00:00+00:00 3      0.0             0.0
                          4      3.0             0.0
                          2      2.0             0.0
                          1      3.0             0.0
2019-03-10 00:00:00+00:00 3     13.0             0.0
                          4     12.0             0.0
                          2     10.0             0.0
                          1      6.0             0.0
2019-03-17 00:00:00+00:00 3      2.0             0.0
                          4      2.0             0.0
                          2      1.0             0.0
                          1      2.0             0.0
2019-03-24 00:00:00+00:00 3      3.0             0.0
                          4      3.0             0.0
                          2      1.0             0.0
                          1      2.0             0.0
2019-03-31 00:00:00+00:00 3      2.0             0.0
                          4      3.0             0.0
                          2      2.0             0.0
                          1      2.0             0.0
2019-04-07 00:00:00+00:00 3      2.0             0.0
                          4      2.0             0.0
                          2      3.0             0.0
                          1      2.0             0.0

Create final df

In [25]:
df_final.head()

num_active_users  num_actions  num_signups
2019-03-03 00:00:00+00:00 3               0.0          0.0          0.0
                          4               3.0         17.0          3.0
                          2               5.0         12.0          2.0
                          1               4.0         12.0          4.0
2019-03-10 00:00:00+00:00 3              28.0        169.0         24.0

In [26]:
df_final_rbs.head()

num_rbs  total_quantity
2019-03-03 00:00:00+00:00 3      0.0             0.0
                          4      3.0             0.0
                          2      2.0             0.0
                          1      3.0             0.0
2019-03-10 00:00:00+00:00 3     13.0             0.0

In [27]:
df_final_merged = pd.merge(df_final, df_final_rbs, left_index=True, right_index=True)

Find average user reportback rate overall

In [28]:
num_signups_peruser = df_mel_merge[df_mel_merge['action_type'] == 'signup'].groupby(['group', 'northstar_id']).size()
num_rbs_peruser = df_rbs_merge.groupby(['group', 'northstar_id'])['post_id'].count()

In [29]:
rb_rates_group_useravg = (num_rbs_peruser / num_signups_peruser).groupby('group').mean()

In [32]:
rb_rates_group_usersavg

group
1    0.338889
2    0.394444
3    0.407018
4    0.530702
dtype: float64